# 1-Norm calculations
After installing the required libraries, the objective is to find the 1-Norm of the qubit Hamiltonian. This Hamiltonian is defined as:
$$
\hat{H}_{\rm qub} = \sum_i^{\mathcal{O}(N^4)} h_i \hat{P}_i
$$
which *could* be obtained from 
$$
\hat{H} = \sum_{p q}^{N} h_{p q}^{} a_p^\dagger a_q^{}+ \frac{1}{2}\sum_{p q r s}^{N} h_{p q r s}^{} a_p^\dagger a_q^\dagger a_r^{} a_s^{}
$$
through a Jordan-Wigner transformation. This 1-Norm is then defined as the sum of absolute values of the coefficients:
$$
||\hat{H}_{\rm qub}|| = \sum_i |h_i|
$$
Instead of doing this JW transformation first, this code gives you the option to directly compute the 1-Norm just from the molecular integrals $h_{pq}$ and $h_{pqrs}$ for any given active space.


In [1]:
from openfermion import MolecularData
from openfermionpyscf import run_pyscf
from openfermion.transforms import get_fermion_operator, jordan_wigner,\
    bravyi_kitaev, JW1norm # JW1norm is the module used to compute qubit-1norms directly from integrals
import module as md # Module to compute some things like Openfermion geometries
import numpy as np

## Initialize molecule and calculation parameters

In [2]:
# Set molecule parameters. You can run the code on any .xyz geometry you want.
# The files have a .txt extension but that shouldn't matter
fname = ['xyz_files/H2nosym.txt','xyz_files/H2COnosym.txt','xyz_files/H10.txt',\
         'xyz_files/C2.txt', 'xyz_files/LiH.txt', 'xyz_files/HLiO.txt', \
         'xyz_files/H2Onosym.txt', 'xyz_files/H14.txt', \
         'xyz_files/hnch2_s0min_dzp.txt', 'xyz_files/hnc3h6_s0min_dzp.txt',\
         'xyz_files/hnc5h10_s0min_dzp.txt', 'xyz_files/hnc7h14_s0min_dzp.txt',\
         'xyz_files/benzene.txt','xyz_files/PCy3.txt','xyz_files/PCy3Cl2Ru.txt'][9]
basis = ['sto-3g', 'cc-pvdz'][1]
multiplicity = 1


# Set calculation parameters.
run_scf = 1
run_mp2 = 0
run_cisd = 0
run_ccsd = 0
run_fci = 0
run_casci = 0

# Choose to consider an active space. 
consider_cas = 1
# If so, define it through the number of active orbitals and active electrons
n_orbitals = 10
n_electrons = 10

if consider_cas:
    localize_cas = 1 # Localize only in the CAS
    only_cas = 1 # Store only the CAS integrals in the molecule object
else:
    localize_cas = 0
    only_cas = 0 

save_tohdf5 = 0 # Choose if you want to save a file with all the integrals
verbose = 0 # Set to 1 if you want a lot of information about the calculation,
            # specifically the time each step takes.

geometry = md.xyz_to_geom(fname) # OpenFermion format for geometry
# Set description of the molecule:
if run_fci:
    description = fname.replace('xyz_files/','').replace('.txt','') +\
        str(basis) + 'FCI'
elif run_casci or consider_cas:
    description = fname.replace('xyz_files/','').replace('.txt','') + str(basis)\
        + 'ne' + str(n_electrons) + 'no' + str(n_orbitals)
else:
    description = fname.replace('xyz_files/','').replace('.txt','') + str(basis)

If you want, you can visualize the molecule with py3Dmol. 

To install py3dmol with pip, run\
`pip install py3Dmol`\
To install py3dmol with conda, run\
`conda install -c conda-forge py3dmol`

In [3]:
import py3Dmol

f = open(fname, 'r')
XYZ_geo = f.read()
N_atoms = md.file_len(fname)
print('number of atoms:',N_atoms)
print('geometry:',XYZ_geo)
p = py3Dmol.view(width=400,height=300)
p.addModel("{} \n\n ".format(N_atoms) + XYZ_geo,'xyz')
p.setStyle({'sphere':{'scale':'0.15'}, 'stick':{'radius':'0.05'}})
p.zoom(7)
p.show()

number of atoms: 11
geometry: N -0.50747536 0.23273591 -0.35428162
C 0.15806969 -0.39299294 0.52445166
H -0.41667784 -0.93344006 1.28336632
H 0.12146052 0.69990131 -1.01477401
C 2.11822521 -1.96552995 0.59759654
C 1.64960563 -0.5076048 0.65409196
H 3.19650489 -2.03195618 0.74332017
H 1.87325068 -2.41217037 -0.36712064
H 1.63694049 -2.56122942 1.37568217
H 2.13235514 0.08285087 -0.13035969
H 1.94444664 -0.0707152 1.61535508



You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## Calculations
Let us first do a calculation with canonical orbitals.

We also compute the fermionic 1-norm, defined as:
$$
\sum_{pq} |h_{pq}| + \sum_{pqrs} |h_{pqrs}|
$$

In [4]:
print('Considering', n_electrons, 'electrons in', n_orbitals, 'orbitals')
print('---------NON-LOCALIZED_ORBITALS---------')
localize = 0
localize_virt = 0 
# Initialize molecule
molecule = MolecularData(
    geometry, basis, multiplicity,
    description=description)

threshold = 1e-10 # If you want, you can set a threshold on the integrals. (Default: 0.)
# Run pyscf.
molecule = run_pyscf(molecule,
                     threshold=threshold,
                     run_scf=run_scf,
                     run_mp2=run_mp2,
                     run_cisd=run_cisd,
                     run_ccsd=run_ccsd,
                     run_fci=run_fci,
                     run_casci=run_casci,
                     n_electrons=n_electrons,
                     n_orbitals=n_orbitals,
                     verbose=verbose,
                     localize=localize,
                     localize_virt=localize_virt,
                     localize_cas=localize_cas,
                     only_cas=only_cas,
                     save_tohdf5=save_tohdf5)

molecular_hamiltonian = molecule.get_molecular_hamiltonian()
# Fermionic 1-Norm:
ferm1norm = np.sum(np.absolute(molecular_hamiltonian.one_body_tensor)) + \
         np.sum(np.absolute(molecular_hamiltonian.two_body_tensor)) + \
         np.absolute(molecular_hamiltonian.constant)
# Calculate the 1-norm after the JW transformation:
testqub1norm = jordan_wigner(molecular_hamiltonian).induced_norm()

# Calculate the 1-norm with the JW1norm module (much faster):
qub1norm = JW1norm(molecular_hamiltonian.constant,
                   molecular_hamiltonian.one_body_tensor,
                   molecular_hamiltonian.two_body_tensor)

Considering 10 electrons in 10 orbitals
---------NON-LOCALIZED_ORBITALS---------
pooling took 0.3451251983642578 seconds
closing pools took 2.384185791015625e-06
adding twobody terms to total took 0.08021330833435059
twobody transformation took 0.5199990272521973
Normal ordering.....
Done normal ordering


In [5]:
print("After finding the canonical molecular orbitals using Hartree-Fock, our molecule",\
      fname.replace('.txt','').replace('xyz_files/',''), "in an active space of", str(n_electrons), \
      "electrons in", str(n_orbitals), "orbitals, has a fermionic 1-norm of", str(ferm1norm), \
      "and a qubit 1-norm of", str(qub1norm) + '. To be sure, we also calculated the 1-norm ' +\
      'after the JW transformation and it was indeed equal to', str(testqub1norm))

After finding the canonical molecular orbitals using Hartree-Fock, our molecule hnc3h6_s0min_dzp in an active space of 10 electrons in 10 orbitals, has a fermionic 1-norm of 457.0517201404062 and a qubit 1-norm of 149.98009313438195. To be sure, we also calculated the 1-norm after the JW transformation and it was indeed equal to 149.98009313437646


## Localized orbitals
We can now try and localize the orbitals, to see if we can lower the 1-norm. Available localization schemes are:
- Boys
- Pipek-Mezey
- Edmiston-Ruedenberg
- IBOs (not sure if they actually work)
- Orthogonal AOs (only full space)

In [6]:
localize = 1
localize_virt = 1 #Choose whether you want to localize the virtual MO's
localizemethod = ['Boys','Pipek-Mezey','ibo','ER'][-1]

# You can also choose whether you want to mix occupied an virtual orbitals when localizing
localize_sep = 0

Orth_AO = 0 # If you want to compute orthogonal AOs, please consider the full space (consider_cas = 0)
print('---------LOCALIZED_ORBITALS---------')

# Initialize molecule
molecule = MolecularData(
    geometry, basis, multiplicity,
    description=description)

# Run pyscf.
molecule = run_pyscf(molecule,
                     threshold=threshold,
                     run_scf=run_scf,
                     run_mp2=run_mp2,
                     run_cisd=run_cisd,
                     run_ccsd=run_ccsd,
                     run_fci=run_fci,
                     run_casci=run_casci,
                     n_electrons=n_electrons,
                     n_orbitals=n_orbitals,
                     verbose=verbose,
                     localize=localize,
                     localizemethod=localizemethod,
                     localize_virt=localize_virt,
                     localize_cas=localize_cas,
                     only_cas=only_cas,
                     localize_sep=localize_sep,
                     Orth_AO=Orth_AO,
                     save_tohdf5=save_tohdf5)

molecular_hamiltonian = molecule.get_molecular_hamiltonian()
# Fermionic 1-Norm:
ferm1norm = np.sum(np.absolute(molecular_hamiltonian.one_body_tensor)) + \
         np.sum(np.absolute(molecular_hamiltonian.two_body_tensor)) + \
         np.absolute(molecular_hamiltonian.constant)
# Calculate the 1-norm after the JW transformation:
testqub1norm = jordan_wigner(molecular_hamiltonian).induced_norm()

# Calculate the 1-norm with the JW1norm module (much faster):
qub1norm = JW1norm(molecular_hamiltonian.constant,
                   molecular_hamiltonian.one_body_tensor,
                   molecular_hamiltonian.two_body_tensor)

---------LOCALIZED_ORBITALS---------
LOCALIZING TOGETHER n_core 11 n_active 10 n_virtual 70
pooling took 0.3245360851287842 seconds
closing pools took 3.5762786865234375e-06
adding twobody terms to total took 0.0799248218536377
twobody transformation took 0.530125617980957
Normal ordering.....
Done normal ordering


In [7]:
print("After finding the", localizemethod, "localized molecular orbitals, our molecule",\
      fname.replace('.txt','').replace('xyz_files/',''), "in an active space of", str(n_electrons), \
      "electrons in", str(n_orbitals), "orbitals, has a fermionic 1-norm of", str(ferm1norm), \
      "and a qubit 1-norm of", str(qub1norm) + '. To be sure, we also calculated the 1-norm ' +\
      'after the JW transformation and it was indeed equal to', str(testqub1norm))

After finding the ER localized molecular orbitals, our molecule hnc3h6_s0min_dzp in an active space of 10 electrons in 10 orbitals, has a fermionic 1-norm of 380.5592349816821 and a qubit 1-norm of 130.1244322897098. To be sure, we also calculated the 1-norm after the JW transformation and it was indeed equal to 130.1244322897144


To really see the effects of localization, you have to consider a big active space. Lets consider the largest formaldimine molecule. I chose an active space of 40 spatial orbitals (you will need approx ~9 GB of memory for this, so if you have a 16GB machine you should be fine. Otherwise, try lowering it to 30 electrons in 30 orbitals)

In [8]:
fname = ['xyz_files/H2nosym.txt','xyz_files/H2COnosym.txt','xyz_files/H10.txt',\
         'xyz_files/C2.txt', 'xyz_files/LiH.txt', 'xyz_files/HLiO.txt', \
         'xyz_files/H2Onosym.txt', 'xyz_files/H14.txt', \
         'xyz_files/hnch2_s0min_dzp.txt', 'xyz_files/hnc3h6_s0min_dzp.txt',\
         'xyz_files/hnc5h10_s0min_dzp.txt', 'xyz_files/hnc7h14_s0min_dzp.txt',\
         'xyz_files/benzene.txt','xyz_files/PCy3.txt','xyz_files/PCy3Cl2Ru.txt'][11]

basis = ['sto-3g', 'cc-pvdz'][0]
multiplicity = 1

consider_cas = 1
# If so, define it through the number of active orbitals and active electrons
nelec = md.count_elec(geometry,basis)
n_orbitals = 40
if n_orbitals > nelec:
    n_electrons = nelec
else:
    n_electrons = n_orbitals


if consider_cas:
    localize_cas = 1 # Localize only in the CAS
    only_cas = 1 # Store only the CAS integrals in the molecule object
else:
    localize_cas = 0
    only_cas = 0 

save_tohdf5 = 0 # Choose if you want to save a file with all the integrals
verbose = 0 # Set to 1 if you want a lot of information about the calculation,
            # specifically the time each step takes.

geometry = md.xyz_to_geom(fname) # OpenFermion format for geometry
# Set description of the molecule:
if run_fci:
    description = fname.replace('xyz_files/','').replace('.txt','') +\
        str(basis) + 'FCI'
elif run_casci or consider_cas:
    description = fname.replace('xyz_files/','').replace('.txt','') + str(basis)\
        + 'ne' + str(n_electrons) + 'no' + str(n_orbitals)
else:
    description = fname.replace('xyz_files/','').replace('.txt','') + str(basis)

Again, you can visualize it with py3Dmol:

In [9]:
import py3Dmol

f = open(fname, 'r')
XYZ_geo = f.read()
N_atoms = md.file_len(fname)
print('number of atoms:',N_atoms)
# print('geometry:',XYZ_geo)
p = py3Dmol.view(width=600,height=500)
p.addModel("{} \n\n ".format(N_atoms) + XYZ_geo,'xyz')
p.setStyle({'sphere':{'scale':'0.15'}, 'stick':{'radius':'0.05'}})
p.zoom(7)
p.show()

number of atoms: 23


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [10]:
print('Considering', n_electrons, 'electrons in', n_orbitals, 'orbitals')
print('---------NON-LOCALIZED_ORBITALS---------')
localize = 0
localize_virt = 0 
# Initialize molecule
molecule = MolecularData(
    geometry, basis, multiplicity,
    description=description)

threshold = 1e-10 # If you want, you can set a threshold on the integrals. (Default: 0.)
# Run pyscf.
molecule = run_pyscf(molecule,
                     threshold=threshold,
                     run_scf=run_scf,
                     run_mp2=run_mp2,
                     run_cisd=run_cisd,
                     run_ccsd=run_ccsd,
                     run_fci=run_fci,
                     run_casci=run_casci,
                     n_electrons=n_electrons,
                     n_orbitals=n_orbitals,
                     verbose=verbose,
                     localize=localize,
                     localize_virt=localize_virt,
                     localize_cas=localize_cas,
                     only_cas=only_cas,
                     save_tohdf5=save_tohdf5)

molecular_hamiltonian = molecule.get_molecular_hamiltonian()
# Fermionic 1-Norm:
ferm1norm = np.sum(np.absolute(molecular_hamiltonian.one_body_tensor)) + \
         np.sum(np.absolute(molecular_hamiltonian.two_body_tensor)) + \
         np.absolute(molecular_hamiltonian.constant)

# We can't do Jordan-Wigner anymore. Calculate the 1-norm with the JW1norm module instead:
qub1norm = JW1norm(molecular_hamiltonian.constant,
                   molecular_hamiltonian.one_body_tensor,
                   molecular_hamiltonian.two_body_tensor)

Considering 32 electrons in 40 orbitals
---------NON-LOCALIZED_ORBITALS---------
Normal ordering.....
Done normal ordering


In [11]:
print("After finding the canonical molecular orbitals using Hartree-Fock, our molecule",\
      fname.replace('.txt','').replace('xyz_files/',''), "in an active space of", str(n_electrons), \
      "electrons in", str(n_orbitals), "orbitals, has a fermionic 1-norm of", str(ferm1norm), \
      "and a qubit 1-norm of", str(qub1norm))

After finding the canonical molecular orbitals using Hartree-Fock, our molecule hnc7h14_s0min_dzp in an active space of 32 electrons in 40 orbitals, has a fermionic 1-norm of 12297.930098954308 and a qubit 1-norm of 2411.9884909975067


Now we try localizing again, this time with Pipek-Mezey (ER takes a bit longer on this size molecule).

In [12]:
localize = 1
localize_virt = 1 #Choose whether you want to localize the virtual MO's
localizemethod = ['Boys','Pipek-Mezey','ibo','ER'][1]

# You can also choose whether you want to mix occupied and virtual orbitals or seperate them when localizing
localize_sep = 0

Orth_AO = 0 # If you want to compute orthogonal AOs, please consider the full space (consider_cas = 0)
print('---------LOCALIZED_ORBITALS---------')

# Initialize molecule
molecule = MolecularData(
    geometry, basis, multiplicity,
    description=description)

# Run pyscf.
molecule = run_pyscf(molecule,
                     threshold=threshold,
                     run_scf=run_scf,
                     run_mp2=run_mp2,
                     run_cisd=run_cisd,
                     run_ccsd=run_ccsd,
                     run_fci=run_fci,
                     run_casci=run_casci,
                     n_electrons=n_electrons,
                     n_orbitals=n_orbitals,
                     verbose=verbose,
                     localize=localize,
                     localizemethod=localizemethod,
                     localize_virt=localize_virt,
                     localize_cas=localize_cas,
                     only_cas=only_cas,
                     localize_sep=localize_sep,
                     Orth_AO=Orth_AO,
                     save_tohdf5=save_tohdf5)

molecular_hamiltonian = molecule.get_molecular_hamiltonian()
# Fermionic 1-Norm:
ferm1norm = np.sum(np.absolute(molecular_hamiltonian.one_body_tensor)) + \
         np.sum(np.absolute(molecular_hamiltonian.two_body_tensor)) + \
         np.absolute(molecular_hamiltonian.constant)

# Calculate the 1-norm with the JW1norm module:
qub1norm = JW1norm(molecular_hamiltonian.constant,
                   molecular_hamiltonian.one_body_tensor,
                   molecular_hamiltonian.two_body_tensor)

---------LOCALIZED_ORBITALS---------
LOCALIZING TOGETHER n_core 16 ntot 56
Normal ordering.....
Done normal ordering


In [13]:
print("After finding the", localizemethod, "localized molecular orbitals, our molecule",\
      fname.replace('.txt','').replace('xyz_files/',''), "in an active space of", str(n_electrons), \
      "electrons in", str(n_orbitals), "orbitals, has a fermionic 1-norm of", str(ferm1norm), \
      "and a qubit 1-norm of", str(qub1norm))

After finding the Pipek-Mezey localized molecular orbitals, our molecule hnc7h14_s0min_dzp in an active space of 32 electrons in 40 orbitals, has a fermionic 1-norm of 2853.6395108307106 and a qubit 1-norm of 558.6300985663509


We see that indeed we can lower the 1-norm significantly! With this code, you can try different localization schemes and molecules yourself.